設定ファイルを読み込む

- ``sandbox/config.toml``
- 質問をカテゴリー型に変換するための設定
- ``[questions]``: 質問の内容を取得するための設定
- ``[choices]``: 質問の回答をカテゴリー型に変換するための設定
- ``choices``は使いまわすことがあるので、``questions``と1対1に対応させず、別々のセクションにした
  - 例: ``regional`` / ``subregional`` / ``good_poor`` / ``agree_disgree`` など

In [ ]:
from pathlib import Path
import tomllib

In [ ]:
fname = Path("../sandbox/config.toml")
with fname.open("rb") as f:
    config = tomllib.load(f)

In [ ]:
config.keys()

In [ ]:
questions = config["questions"]
questions.keys()

In [ ]:
choices = config["choices"]
choices.keys()

In [ ]:
volumes = config["volumes"]
volumes.keys()

In [ ]:
volumes